In [1]:
import pandas as pd
import numpy as np
import random

In [5]:
#read data
pathology=pd.read_csv('open_tggates_pathology.csv' , encoding='unicode_escape')

In [7]:
#chosen Orgen
pathology=pathology[pathology['ORGAN']=='Kidney']
#no chip data means that there is no gene expression for this sample
pathology=pathology[pathology['BARCODE']!='No ChipData']
##specefic compounds
#it musgt be 41 drugs 


kidney_drugs = ["acetaminophen", "acetazolamide", "allopurinol", "allyl alcohol", "bromobenzene", "bromoethylamine", "bucetin", "caffeine", "captopril", "carboplatin", "cephalothin", "ciprofloxacin", "cisplatin", "clofibrate", "cyclophosphamide", "cyclosporine A", "desmopressin acetate", "doxorubicin", "enalapril", "erythromycin ethylsuccinate", "ethinylestradiol", "ethionine", "gentamicin", "hexachlorobenzene", "imipramine", "indomethacin", "ketoconazole", 
                "lomustine", "methyltestosterone", "monocrotaline", "nitrofurantoin", "omeprazole", "phenacetin", "phenylanthranilic acid", "phenylbutazone", "puromycin aminonucleoside", "rifampicin", "thioacetamide", "triamterene", "valproic acid"]
pathology=pathology[pathology.COMPOUND_NAME.isin(kidney_drugs)]


In [8]:
#special pathways are related to nephrotoxicity 
keywords = ['Dilatation, cystic', 'Necrosis','Dilatation', 'Regeneration',
            'Cast,hyaline', 'Change, basophilic', 'Cellular infiltration, lymphocyte','Cyst']
            
pathology=pathology[pathology['FINDING_TYPE'].isin(keywords)]
pathology=pathology[pathology['DOSE_LEVEL']!='Control']

In [5]:
chosen_hours=['24 hr','6 hr','3 hr' ,'9 hr']
pathology=pathology[pathology['SACRIFICE_PERIOD'].isin(chosen_hours)]

In [17]:
unique_drugs = []
#my goal was to  remove if we see any redandant sample 
# Iterate through each row in the DataFrame
##if multiple samples accur
for index, row in pathology.iterrows():
    compound = row['COMPOUND_NAME']
    dose=row['DOSE_LEVEL']
    period=row['SACRIFICE_PERIOD']
    path = row['FINDING_TYPE']

    
    
    # Check if the combination is unique
    if (compound,path,dose,period) not in unique_drugs:

        unique_drugs.append((compound,path,dose,period))
        
new_filtered_pathology=pd.DataFrame(unique_drugs,columns=['COMPOUND_NAME','FINDING_TYPE','dose','period'])

In [20]:
#I only take 2 hours , becuase the toxicogenomic data contain 24 hours compounds 
new_filtered_pathology=new_filtered_pathology[new_filtered_pathology['period']=='24 hr']
new_filtered_pathology['dose'] = new_filtered_pathology['dose'].replace({
    'Middle': 'M',
    'High': 'H',
    'Low': 'L'
})


In [25]:
new_filtered_pathology.index=new_filtered_pathology['COMPOUND_NAME']+' '+new_filtered_pathology['dose']

In [26]:
label_columns = ['Necrosis', 'Dilatation', 'Dilatation, cystic', 'Cast,hyaline',
                 'Regeneration', 'Cyst', 'Cellular infiltration, lymphocyte',
                 'Change, basophilic']

data_to_gather = {col: set() for col in label_columns}


for index, row in new_filtered_pathology.iterrows():
    finding_type = row['FINDING_TYPE']
    compound_name = row['COMPOUND_NAME']
    dose = row['dose']
    
    if finding_type in label_columns:
        # Add a tuple of (compound_name, dose) to the appropriate set
        data_to_gather[finding_type].add((compound_name, dose))

Final_data = pd.DataFrame(0, index=new_filtered_pathology.index, columns=label_columns)

# Iterate over the dataframe again to set the multi-label classification columns
for index, row in new_filtered_pathology.iterrows():
    compound_name = row['COMPOUND_NAME']
    dose = row['dose']
    
    for finding_type in label_columns:
        if (compound_name, dose) in data_to_gather[finding_type]:
            Final_data.at[index, finding_type] = 1


In [27]:
Final_data['COMPOUND_NAME'] = [
    " ".join(compound.split()[0:2]) if len(compound.split()) > 2 else compound.split()[0]
    for compound in Final_data.index
]

Final_data['DOSE'] = [compound.split()[2] if len(compound.split()) > 2 else compound.split()[1] for compound in Final_data.index]
Final_data=Final_data.drop_duplicates()
